In [1]:
import os
os.chdir('..')


In [2]:
from IPython.display import clear_output
from kg_rag.utility import *

clear_output()


In [3]:

def generate_response(question, llm, kg_rag_flag, evidence_flag=False, temperature=0):
    CHAT_MODEL_ID = llm
    CHAT_DEPLOYMENT_ID = llm
    
    if kg_rag_flag:
        SYSTEM_PROMPT = system_prompts["KG_RAG_BASED_TEXT_GENERATION"]
        CONTEXT_VOLUME = int(config_data["CONTEXT_VOLUME"])
        QUESTION_VS_CONTEXT_SIMILARITY_PERCENTILE_THRESHOLD = float(config_data["QUESTION_VS_CONTEXT_SIMILARITY_PERCENTILE_THRESHOLD"])
        QUESTION_VS_CONTEXT_MINIMUM_SIMILARITY = float(config_data["QUESTION_VS_CONTEXT_MINIMUM_SIMILARITY"])
        VECTOR_DB_PATH = config_data["VECTOR_DB_PATH"]
        NODE_CONTEXT_PATH = config_data["NODE_CONTEXT_PATH"]
        SENTENCE_EMBEDDING_MODEL_FOR_NODE_RETRIEVAL = config_data["SENTENCE_EMBEDDING_MODEL_FOR_NODE_RETRIEVAL"]
        SENTENCE_EMBEDDING_MODEL_FOR_CONTEXT_RETRIEVAL = config_data["SENTENCE_EMBEDDING_MODEL_FOR_CONTEXT_RETRIEVAL"]
        vectorstore = load_chroma(VECTOR_DB_PATH, SENTENCE_EMBEDDING_MODEL_FOR_NODE_RETRIEVAL)
        embedding_function_for_context_retrieval = load_sentence_transformer(SENTENCE_EMBEDDING_MODEL_FOR_CONTEXT_RETRIEVAL)
        node_context_df = pd.read_csv(NODE_CONTEXT_PATH)
        context = retrieve_context(question, vectorstore, embedding_function_for_context_retrieval, node_context_df, CONTEXT_VOLUME, QUESTION_VS_CONTEXT_SIMILARITY_PERCENTILE_THRESHOLD, QUESTION_VS_CONTEXT_MINIMUM_SIMILARITY, evidence_flag)
        enriched_prompt = "Context: "+ context + "\n" + "Question: " + question
        question = enriched_prompt
    else:
        SYSTEM_PROMPT = system_prompts["PROMPT_BASED_TEXT_GENERATION"]
    
    output = get_GPT_response(question, SYSTEM_PROMPT, CHAT_MODEL_ID, CHAT_DEPLOYMENT_ID, temperature=temperature)
    stream_out(output)


In [4]:

LLM_TO_USE = 'gpt-4'
TEMPERATURE = config_data["LLM_TEMPERATURE"]


## Question 1:

In [6]:
question = 'Are there any latest drugs used for weight management in patients with Bardet-Biedl Syndrome?'


### With KG-RAG

In [81]:
KG_RAG_FLAG = True
EDGE_EVIDENCE_FLAG = True #Used only when KG_RAG_FLAG=True

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, evidence_flag=EDGE_EVIDENCE_FLAG, temperature=TEMPERATURE)


Yes, the compound Setmelanotide is used to treat Bardet-Biedl syndrome. This information is sourced from ChEMBL and DrugCentral.



### Without KG-RAG

In [82]:
KG_RAG_FLAG = False

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, temperature=TEMPERATURE)


As of my knowledge up to date, there are no specific drugs designed for weight management in patients with Bardet-Biedl Syndrome. The treatment generally involves managing the symptoms and complications. However, any new developments would be best advised by a healthcare professional.



## Question 2:

In [98]:
question = 'Is it PNPLA3 or HLA-B that has a significant association with the disease liver benign neoplasm?'


### With KG-RAG

In [100]:
KG_RAG_FLAG = True
EDGE_EVIDENCE_FLAG = True #Used only when KG_RAG_FLAG=True

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, evidence_flag=EDGE_EVIDENCE_FLAG, temperature=TEMPERATURE)


PNPLA3 has a more significant association with the disease liver benign neoplasm, as indicated by its lower GWAS p-value of 4e-14 compared to HLA-B's p-value of 2e-08. The provenance of this association is GWAS.



### Without KG-RAG

In [99]:
KG_RAG_FLAG = False

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, temperature=TEMPERATURE)


It is PNPLA3 that has a significant association with the disease liver benign neoplasm.



## Other questions:

In [13]:

question = 'Does drug dependence have any genetic factors? Do you have any statistical evidence from trustworthy sources for this?'


In [14]:
KG_RAG_FLAG = True
EDGE_EVIDENCE_FLAG = True 

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, evidence_flag=EDGE_EVIDENCE_FLAG, temperature=TEMPERATURE)


Yes, drug dependence does have genetic factors. This is evidenced by the association of drug dependence with genes KAT2B and SLC25A16. The statistical evidence comes from Genome-Wide Association Studies (GWAS), with p-values of 4e-10 and 1e-09 respectively, indicating a statistically significant association.



In [15]:
KG_RAG_FLAG = False

generate_response(question, LLM_TO_USE, KG_RAG_FLAG, temperature=TEMPERATURE)


Yes, drug dependence does have genetic factors. According to the National Institute on Drug Abuse, genetics account for about 40-60% of a person's vulnerability to drug addiction.

